In [1]:
import requests
import csv, re, os
import json
import pandas as pd
import numpy as np
from time import sleep

In [6]:
answers = pd.read_csv('zhihu/zhihu.csv',dtype=object)
savefile = 'zhihu/zhihu_comments.csv'


In [7]:
def get_author_info(author_url):
    res = requests.get(author_url)
    if gender := re.search(r'<meta itemProp="gender" content="(\w+)"',res.text, re.IGNORECASE):
        gender = gender.group(1)
    if followers := re.search(r'<meta itemProp="zhihu:followerCount" content="(\d*)"',res.text, re.IGNORECASE):
        followers = int(followers.group(1))
    if location := re.search(r'"ipInfo":"IP 属地(.*)","vip',res.text, re.IGNORECASE):
        location = location.group(1)
    if following := re.search(r'关注了</div><strong class="NumberBoard-itemValue" title="(\d+)">',res.text, re.IGNORECASE):
        following = int(following.group(1))
    return gender, followers, location, following

In [8]:
with open(savefile, 'a+') as f:
    header = ['title','question_url','answer_id','answer_content','comment_id','created_time','comment_content','comment_upvote',
              'child_comments_count','author_info','author_gender']
    writer = csv.DictWriter(f, delimiter=',', lineterminator='\n',fieldnames=header)
    if not os.path.getsize(savefile):
        writer.writeheader()
        
    for index, row in answers.iterrows():
        if type(row['answer_id']) == str:
            continue
        answer_id = re.search(r'answer/(\d+)',row['url'], re.IGNORECASE).group(1)
        answers.loc[index,'answer_id'] = answer_id  
        page = 0
        while(1):
            res = requests.get(f'https://www.zhihu.com/api/v4/answers/{answer_id}/root_comments?order=normal&limit=20&offset={page}&status=open')
            if ('ForbiddenError' in res.text) or('NotFoundError' in res.text):
                break
            data = json.loads(res.text)
            if len(data['data']) == 0: 
                break

            for comment in data['data']:
                try:
                    child_comments_count = comment['child_comment_count']
                except:
                    child_comments_count = np.NaN
                writer.writerow({'title':row['title'],'question_url':row['question_url'],'answer_id':str(answer_id), 'answer_content':row['texts'],
                                 'comment_id': str(comment['id']), 'created_time': comment['created_time'],'comment_content': comment['content'],
                                 'comment_upvote': comment['vote_count'],'child_comments_count': child_comments_count,
                                'author_info': comment['author']['member']['url'],'author_gender':comment['author']['member']['gender']
                                })
            page+=len(data['data'])
            sleep(5)
        print(answer_id)
        answers.loc[index,'root_comment'] = page
        answers.to_csv('zhihu/zhihu.csv',index=False)
        sleep(5)


2862779850
2465243949
2012040422
1968180894
1948692914
2083467550
2989497420
2098747386
2935108614
2897494466
3021413162
1439796936
2175403728
1837863688
1851983351
1852078430
1851972081
1853024024
1852153351
1858691945
1857563150
1855121240
1854636555
1867383533
1853019684
1853009098
1853286616
1852033351
1857083795
1854021741
1858341167
1853051541
1860500987
1852546100
1855010843
1854137212
1855022603
1852850342
1852706257
1852786708
1852859610
1855033696
1856748761
1855427520
1854941372
1855283068
1855298620
1853671357
1857874671
1854538185
1852847814
1854784585
1857652738
1852210856
1855287738
1857537681
1852034642
1852652820
1853681150
1853157683
1852078750
1859341277
1855882611
1852258320
1853372048
1852642262
1852771192
1869708997
1854265923
1855730098
1859432524
1852767262
1853225156
1852956059
1858030583
1852820402
1854045277
1853512342
1858465630
1868023189
1852637466
1853642738
1858379726
1852876756
1856648933
1854332829
1858078408
1853770949
1859846626
1854323323
1853353557